In [ ]:
import pandas as pd

In [ ]:
# archivo con reportes
reportsDf = pd.read_csv('reports.csv',encoding="ISO-8859-1")

In [ ]:
# obtener entidades tanto LLT como mediante fuzzy que se van a utilizar
LLT_entidades = pd.read_csv("entidades.csv")

In [ ]:
#obtener muestra
subset=reportsDf.sample(8000)
subset= subset.dropna()
subset = subset.drop_duplicates('SYMPTOM_TEXT')
subset.to_csv("subset_code_v1.0.csv",encoding="ISO-8859-1")

In [ ]:
#Proceso para crear los patrones que se requieren etiquetar
def createPatterns(finalDataframe):
    patterns = []
    for index, row in finalDataframe.iterrows():
        
        patternSegments =  row['entities'].split(" ")
        patternSegmentsArray = []
        for pattern in patternSegments:
            patternSegmentsArray.append({"LOWER": pattern})
        patterns.append({"label": 'MedDRA',"pattern":patternSegmentsArray})

        patterns.append({
            "label": "MedDRA",
            "pattern": [{"ORTH": row["entities"].lower()}],
        })
       
    return patterns

In [ ]:
# Invocar metodo para crear patrones
patterns = createPatterns(LLT_entidades)

In [ ]:
from pprint import pprint

# uso de gpu
spacy.prefer_gpu()

#cargar modelo a utilizar
nlp = spacy.load("en_core_web_sm")

#utilizar componente entity ruler
ruler = nlp.add_pipe("entity_ruler")

#agregar componente
with nlp.select_pipes(enable="tagger"):
    ruler.add_patterns(patterns)

In [ ]:

# campos que se colocan en archivo csv que sirve de salida
field_names=["text","entities"]

start = timer()
with open('train_dataset.csv', 'a', newline='', encoding="ISO-8859-1") as f_object:
      

        dictwriter_object = DictWriter(f_object, fieldnames=field_names)
        

        for index, row in subset.iterrows():
            entities = []
            doc = nlp(row['SYMPTOM_TEXT'])
            entityDict = {"links":{},"entities":[]}
            for ent in doc.ents:

                if ent.label_ == "MedDRA":
                
                    for c in kb.get_alias_candidates(ent.text):
                        key = "("+str(ent.start_char)+","+str(ent.end_char)+")"
                        entityDict["links"][key] = {str(c.entity_):1.0}
                    entityDict["entities"].append((ent.start_char,ent.end_char,str(ent.label_)))
                
            dictwriter_object.writerow({"text":row['SYMPTOM_TEXT'],"entities":entityDict})
end = timer()
print("Time per batch")
print(end - start) 